# Pseudo-code for project 2

OBJECTIVE: Find parameters of shots that lead to goals using exhaustive parameter scan

In [3]:
import numpy as np
import matplotlib.pyplot as plt


Should be at least 3 pages long

First, define a shooting alogthim function in terms of 
    rho = air density
    g = Accl due to gravity
    m = mass 
    D = diameter 
    v0 = speed 
    omega = spin (the axis of rotation is perpentidular with the ground)
    phi = azimuthal angle (horivantal angle)
    Theta = elevation angle (vertical angle)
    
[inspiration can be taken from the baseball homework, hw 7]

Second, set up enviroemnts with proper dimensions for the two different types of kicks, including the goal and its target area. 
    1. free kick 
    2. corner kick

Third, run shooting alogthim through range of paramters. 

In [20]:
#parameters for free kick 
rho = 1.2     # kg m^-3
g= 9.81     # kg m s^-2
m = 0.436    # kg
D = 0.218     # m
v0 = np.arange(24, 30.1, .1)   # m s^-1
omega = np.arange(0, 75.6, .2)   # rad s^-1
phi = np.arange(-0.0349066, 0.0872665, .001)  # rad [-2 deg to 5 deg]
theta = np.arange(.174533, 0.349066, .001) # rad [10 deg to 20 deg]

In [23]:
#parameters for corner kick
rho = 1.2     # kg m^-3
g= 9.81     # kg m s^-2
m = 0.436    # kg
D = 0.218     # m
v0 = np.arange(24, 30.1, .1)   # m s^-1
omega = np.arange(0, 75.6, .2)   # rad s^-1
phi = np.arange(-0.0698132, 0.0698132, .001)  # rad [-4 deg to 4 deg]
theta = np.arange(0.349066, 0.523599, .001) # rad [20 deg to 30 deg]

In [21]:
theta

array([ 0.174533,  0.175533,  0.176533,  0.177533,  0.178533,  0.179533,
        0.180533,  0.181533,  0.182533,  0.183533,  0.184533,  0.185533,
        0.186533,  0.187533,  0.188533,  0.189533,  0.190533,  0.191533,
        0.192533,  0.193533,  0.194533,  0.195533,  0.196533,  0.197533,
        0.198533,  0.199533,  0.200533,  0.201533,  0.202533,  0.203533,
        0.204533,  0.205533,  0.206533,  0.207533,  0.208533,  0.209533,
        0.210533,  0.211533,  0.212533,  0.213533,  0.214533,  0.215533,
        0.216533,  0.217533,  0.218533,  0.219533,  0.220533,  0.221533,
        0.222533,  0.223533,  0.224533,  0.225533,  0.226533,  0.227533,
        0.228533,  0.229533,  0.230533,  0.231533,  0.232533,  0.233533,
        0.234533,  0.235533,  0.236533,  0.237533,  0.238533,  0.239533,
        0.240533,  0.241533,  0.242533,  0.243533,  0.244533,  0.245533,
        0.246533,  0.247533,  0.248533,  0.249533,  0.250533,  0.251533,
        0.252533,  0.253533,  0.254533,  0.255533, 

Objectives
 1. talk about drag and lift coeffcients of ball
 2. simulate trajectories of free kick and corner kick
 3. plot in 2D and 3D and desricbe paramters of trajectories
 4. Talk in depth about at one goal scoring shot for each: include parameters plots and a plot where omega = 0 to show influence of spin.

trajectory of free kick should stop calculating when
  1. ball reaches goal line
  2. ball touches ground
  3. ball touches wall

trajectory of corner kick should stop calculating when
  1. ball reaches goal line
  2. ball touches ground

Possbily use 3D-plotting to show trajectories of shots. (especally goal-making shots). guidense can come from 14_PDEs_2.ipynb

include refences and acknolegdments 
